

## 01_generate_parquet_data.ipynb

In this we simple generate some test data in /var/tmp/testdata you can use later. Note this folder is not persistent on OSX, but typically persistent on Linux. Act accordingly!


In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

"""
Kamiwaza intro:

In this notebook, we simply generate some sample data. It will create a set of
parquet files with a super rudimentary fake customer dataset in the OUTPUT_FOLDER
"""
# will try to create if not exists
OUTPUT_FOLDER='/var/tmp/testdata'

"""generate_parquet.py

    Usage: python generate_parquet.py /var/tmp/testdata (which is the default)
    
    Generates parquet files with a very rudimentary scheme to showcase parquet-based ingestion
    """
# Function to generate customers data
def generate_customers(num_customers=400):
    first_names = ["John", "Jane", "Alice", "Bob", "Carol", "Dave", "Eve", "Frank", "Grace", "Hank"]
    last_names = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Miller", "Davis", "Garcia", "Rodriguez", "Wilson"]

    customers = pd.DataFrame({
        "customer_id": range(1, num_customers + 1),
        "first_name": np.random.choice(first_names, num_customers),
        "middle_initial": np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), num_customers),
        "last_name": np.random.choice(last_names, num_customers)
    })
    return customers

# Function to generate orders data
def generate_orders(num_orders=5000, num_customers=400, start_date="2024-01-01", end_date="2024-01-03"):
    dates = pd.date_range(start_date, end_date).tolist()
    orders = pd.DataFrame({
        "order_id": range(1, num_orders + 1),
        "customer_id": np.random.randint(1, num_customers + 1, num_orders),
        "order_date": np.random.choice(dates, num_orders)
    })
    return orders

# Function to generate order items data
def generate_order_items(order_data: pd.DataFrame, num_order_items: int = 10000, num_orders: int = 5000) -> pd.DataFrame:
    """
    Generate a DataFrame containing order items with random data.

    Args:
        order_data (pd.DataFrame): The DataFrame containing the orders data.
        num_order_items (int): The number of order items to generate. Defaults to 10000.
        num_orders (int): The number of orders to reference. Defaults to 5000.

    Returns:
        pd.DataFrame: A DataFrame with the generated order items data.
    """
    item_names = [f"Widget {i}" for i in range(1, 11)]
    order_items = pd.DataFrame({
        "item_id": range(1, num_order_items + 1),
        "order_id": np.random.randint(1, num_orders + 1, num_order_items),
        "item_name": np.random.choice(item_names, num_order_items),
        "quantity": np.random.randint(1, 6, num_order_items),
    })
    # Add the date from the corresponding order to each order item
    order_items = order_items.merge(order_data[['order_id', 'order_date']], on='order_id', how='left')
    return order_items

if __name__ == "__main__":
    # Generating data
    customers = generate_customers()
    orders = generate_orders()
    order_items = generate_order_items(order_data=orders)

    # Save to Parquet files
    import sys
    
    # Set the default output folder, but allow for an argument to override it
    output_folder = OUTPUT_FOLDER
    os.makedirs(output_folder, exist_ok=True)

    # Customers - unpartitioned
    customers_file = os.path.join(output_folder, "customers.parquet")
    customers.to_parquet(customers_file, index=False)

    # Orders - partitioned by date
    orders_folder = os.path.join(output_folder, "orders")
    for date, orders_group in orders.groupby(orders['order_date'].dt.date):
        orders_partition_folder = os.path.join(orders_folder, f"date={date}")
        os.makedirs(orders_partition_folder, exist_ok=True)
        orders_group.drop(columns="order_date").to_parquet(os.path.join(orders_partition_folder, "data.parquet"), index=False)

    # Order Items - partitioned by date
    order_items_folder = os.path.join(output_folder, "order_items")
    for date, order_items_group in order_items.groupby(order_items['order_date'].dt.date):
        order_items_partition_folder = os.path.join(order_items_folder, f"date={date}")
        os.makedirs(order_items_partition_folder, exist_ok=True)
        order_items_group.drop(columns="order_date").to_parquet(os.path.join(order_items_partition_folder, "data.parquet"), index=False)

    print(f"Data generated in: {output_folder}")
